In [28]:
import pandas as pd 

X_train = pd.read_csv('/kaggle/working/X_train_fd003.csv')
y_train = pd.read_csv('/kaggle/working/y_train_fd003.csv')
X_test = pd.read_csv('/kaggle/working/X_test_fd003.csv')
y_test = pd.read_csv('/kaggle/working/y_test_fd003.csv')


X_train.shape , y_train.shape , X_test.shape , y_test.shape


((24720, 16), (24720, 1), (100, 16), (100, 1))

In [29]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import sklearn
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

2025-09-23 11:52:38.799926: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758628359.134364      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758628359.231572      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [31]:
# Convert DataFrames to NumPy arrays first
X_train_array = X_train.values  # or X_train.to_numpy()
X_test_array = X_test.values    # or X_test.to_numpy()

# Now reshape the arrays
X_train_3d = X_train_array.reshape((X_train_array.shape[0], 1, X_train_array.shape[1]))
X_test_3d = X_test_array.reshape((X_test_array.shape[0], 1, X_test_array.shape[1]))

print("X_train_3D shape:", X_train_3d.shape)
print("X_test_3D shape:", X_test_3d.shape)

X_train_3D shape: (24720, 1, 16)
X_test_3D shape: (100, 1, 16)


In [33]:
X_train.shape

(24720, 16)

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(X_train_3d.shape[1], X_train_3d.shape[2])))
model.add(BatchNormalization())
model.add(Dropout(0.1))


model.add(LSTM(64, return_sequences=True, activation='tanh'))
model.add(Dropout(0.1))


model.add(LSTM(64, return_sequences=True, activation='tanh'))
model.add(Dropout(0.1))


# Third LSTM Layer (last one without return_sequences)
model.add(LSTM(32, activation='tanh'))
model.add(Dropout(0.1))

# Fully Connected Layers
model.add(Dense(64, activation='relu'))
model.add(Dense(1))   # Regression output (e.g., RUL prediction)

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train_3d, y_train,
    validation_data=(X_test_3d, y_test),
    epochs=100,
    batch_size=64,
    callbacks=[es]   #early stopping
)


I0000 00:00:1758628988.900260      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1758628988.900891      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Epoch 1/100


I0000 00:00:1758628997.974598     181 cuda_dnn.cc:529] Loaded cuDNN version 90300


387/387 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - loss: 6091.1899 - mae: 65.3579 - val_loss: 935.7333 - val_mae: 24.4421
Epoch 2/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 324.4308 - mae: 13.0290 - val_loss: 347.7530 - val_mae: 13.9804
Epoch 3/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 261.9884 - mae: 11.5836 - val_loss: 403.2576 - val_mae: 15.1023
Epoch 4/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 246.1106 - mae: 11.1502 - val_loss: 300.3574 - val_mae: 13.3342
Epoch 5/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 232.7362 - mae: 10.9004 - val_loss: 516.8672 - val_mae: 18.0751
Epoch 6/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 229.8161 - mae: 10.7243 - val_loss: 310.2219 - val_mae: 13.2580
Epoch 7/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 230.4429 - mae: 10.7957 - val_loss: 313.8040 - val_mae: 13.5964
Epoch 8/100
387/387 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 225.4541 - mae: 10.6822 - val_loss: 332.4904 - val_mae: 13.6403
Epoch 9/1

In [36]:
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score

y_pred = model.predict(X_test_3d)

mae = mean_absolute_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

accuracy_percent = r2 * 100

print("Accuracy (%):", accuracy_percent)
print("MAE:", mae)
print("R2 Score:", r2)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step
Accuracy (%): 82.65601525121171
MAE: 13.061365175247193
R2 Score: 0.8265601525121171


# save model

In [37]:
model.save("fd003_model.keras")